In [1]:
# !git clone https://github.com/huggingface/transformers.git
# !pip install ./transformers
# !pip install transformers[torch]==4.3
# !pip install -U nlp
!pip install transformers
!pip install sentencepiece
# !pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 94.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.7 MB/s eta 0:00:00


In [2]:
# download
!pip install transformers
!pip install simplet5

# import
import re
import json
import torch

import random
import pandas as pd
from tqdm import tqdm
from simplet5 import SimpleT5
from torch.utils.data import Dataset
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 KB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.2/829.2 KB 49.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 KB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 54.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for simplet5: filename=simplet5-0.1.4-py3-none-any.whl size=6873 sha256=6b2378f123a9f9658657242a33b831fb76afc73bbe139e33fef14002373d0545
  Stored in directory: /root/.cache/pip/wh

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## BERT

In [4]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [5]:
# # Download data
# import requests
# request = requests.get("https://drive.google.com/uc?export=download&id=1wHt8PsMLsfX5yNSqrt2fSTcb8LEiclcf")
# with open("data.zip", "wb") as file:
#     file.write(request.content)

# # Unzip data
# import zipfile
# with zipfile.ZipFile('data.zip') as zip:
#     zip.extractall('data')

In [6]:
# Data load function
def load_sentiment_dataset(random_seed = 1, file_path="/content/drive/MyDrive/training.1600000.processed.noemoticon.csv"):
    # load dataset and sample 10k reviews.
    df = pd.read_csv(file_path, encoding='ISO-8859-1', header=None)
    df = df[[0, 5]]
    df.columns = ['label', 'text']
    df = df.sample(10000, random_state=1)
    
    def pick_first_n_words(string, max_words=250): # tried a few max_words, kept 250 as max tokens was < 512
        split_str = string.split()
        return " ".join(split_str[:min(len(split_str), max_words)])

    df['text'] = df['text'].apply(lambda x: pick_first_n_words(x))
    map_label = {0:'negative', 4: 'positive'}
    df['label'] = df['label'].apply(lambda x: map_label[x])
    
    # divide into test and train
    X_train, X_test, y_train, y_test = \
              train_test_split(df['text'].tolist(), df['label'].tolist(),
              shuffle=True, test_size=0.05, random_state=random_seed, stratify=df['label'])
    
    # transform to pandas dataframe
    train_data = pd.DataFrame({'source_text': X_train, 'target_text': y_train})    
    test_data = pd.DataFrame({'source_text': X_test, 'target_text': y_test})    

    # return
    return train_data, test_data

In [7]:
train_df, test_df = load_sentiment_dataset(0)


for i in range(len(train_df['target_text'])):
  if train_df.loc[i, 'target_text'] == 'positive':
    train_df.loc[i, 'target_text'] = 1
  else:
    train_df.loc[i, 'target_text'] = 0

for i in range(len(test_df['target_text'])):
  if test_df.loc[i, 'target_text'] == 'positive':
    test_df.loc[i, 'target_text'] = 1
  else:
    test_df.loc[i, 'target_text'] = 0

print(train_df)
print(test_df)

                                            source_text target_text
0     @_CorruptedAngel sorry I missed you, hope you ...           1
1     Dont get me wrong. Im ecstatic that u guys hav...           1
2     Damn! My tumblarity s really decreasing! Don't...           0
3     A non &quot;hobby&quot; related emergency had ...           0
4     I got acquainted with a laser etching contact ...           1
...                                                 ...         ...
9495  Tryinq to find jade on here, not sucessful so far           0
9496  @iamlikesounds how do you know? She was one of...           0
9497                                    headacheeeeeeee           0
9498  @karleemay I left last night at around 9pm. He...           0
9499  oh i forgot to tweet. haha. by the way i need ...           1

[9500 rows x 2 columns]
                                           source_text target_text
0    OK...time to stop twitpicing and posterousing ...           1
1    @Kristens_Game pink 

In [8]:
#  # Load data and set labels
# data_complaint = pd.read_csv('data/complaint1700.csv')
# data_complaint['label'] = 0
# data_non_complaint = pd.read_csv('data/noncomplaint1700.csv')
# data_non_complaint['label'] = 1

# # Concatenate complaining and non-complaining data
# data = pd.concat([data_complaint, data_non_complaint], axis=0).reset_index(drop=True)

# # Drop 'airline' column
# data.drop(['airline'], inplace=True, axis=1)

# # Display 5 random samples
# print(data.loc[553])
# data.sample(5)

In [9]:
# from sklearn.model_selection import train_test_split

# X = data.tweet.values
# y = data.label.values

# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=2020)

In [10]:
# # Load test data
# test_data = pd.read_csv('data/test_data.csv')

# # Keep important columns
# test_data = test_data[['id', 'tweet']]

# # Display 5 samples from the test data
# test_data.sample(5)

In [11]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


In [12]:
# import nltk
# # Uncomment to download "stopwords"
# nltk.download("stopwords")
# from nltk.corpus import stopwords

# def text_preprocessing(s):
#     """
#     - Lowercase the sentence
#     - Change "'t" to "not"
#     - Remove "@name"
#     - Isolate and remove punctuations except "?"
#     - Remove other special characters
#     - Remove stop words except "not" and "can"
#     - Remove trailing whitespace
#     """
#     s = s.lower()
#     # Change 't to 'not'
#     s = re.sub(r"\'t", " not", s)
#     # Remove @name
#     s = re.sub(r'(@.*?)[\s]', ' ', s)
#     # Isolate and remove punctuations except '?'
#     s = re.sub(r'([\'\"\.\(\)\!\?\\\/\,])', r' \1 ', s)
#     s = re.sub(r'[^\w\s\?]', ' ', s)
#     # Remove some special characters
#     s = re.sub(r'([\;\:\|•«\n])', ' ', s)
#     # Remove stopwords except 'not' and 'can'
#     s = " ".join([word for word in s.split()
#                   if word not in stopwords.words('english')
#                   or word in ['not', 'can']])
#     # Remove trailing whitespace
#     s = re.sub(r'\s+', ' ', s).strip()
    
#     return s

In [13]:
import nltk
# Uncomment to download "stopwords"
nltk.download("stopwords")
from nltk.corpus import stopwords

def text_preprocessing(text):
    """
    - Remove entity mentions (eg. '@united')
    - Correct errors (eg. '&amp;' to '&')
    param    text (str): a string to be processed.
    return   text (Str): the processed string.
    """
    # Remove '@name'
    text = re.sub(r'(@.*?)[\s]', ' ', text)

    # Replace '&amp;' with '&'
    text = re.sub(r'&amp;', '&', text)

    # # Remove stopwords except 'not' and 'can'
    # text = " ".join([word for word in text.split()
    #               if word not in stopwords.words('english')
    #               or word in ['not', 'can']])

    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [14]:
# Print sentence 0
print('Original: ', train_df['source_text'][0])
print('Processed: ', text_preprocessing(train_df['source_text'][0]))

Original:  @_CorruptedAngel sorry I missed you, hope you have a great time out x
Processed:  sorry I missed you, hope you have a great time out x


In [15]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Create a function to tokenize a set of texts
def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in data:
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        encoded_sent = tokenizer.encode_plus(
            text=text_preprocessing(sent),  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,                  # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True      # Return attention mask
            )
        
        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [16]:
# Concatenate train data and test data
# all_tweets = np.concatenate([data.tweet.values, test_data.tweet.values])
all_tweets = np.concatenate([train_df.source_text.values, test_df.source_text.values])

# Encode our concatenated data
# encoded_tweets = [tokenizer.encode(sent, add_special_tokens=True) for sent in all_tweets]
encoded_tweets = [tokenizer.encode(sent, add_special_tokens=True) for sent in all_tweets]

# Find the maximum length
max_len = max([len(sent) for sent in encoded_tweets])
print('Max length: ', max_len)

Max length:  71


In [17]:
# Specify `MAX_LEN`
MAX_LEN = 64

# Print sentence 0 and its encoded token ids
token_ids = list(preprocessing_for_bert([train_df['source_text'][0]])[0].squeeze().numpy())
print('Original: ', train_df['source_text'][0])
print('Token IDs: ', token_ids)

# Run function `preprocessing_for_bert` on the train set and the validation set
print('Tokenizing data...')
train_inputs, train_masks = preprocessing_for_bert(train_df.source_text.values)
val_inputs, val_masks = preprocessing_for_bert(test_df.source_text.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Original:  @_CorruptedAngel sorry I missed you, hope you have a great time out x
Token IDs:  [101, 3374, 1045, 4771, 2017, 1010, 3246, 2017, 2031, 1037, 2307, 2051, 2041, 1060, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Tokenizing data...


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2251: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [18]:
print(train_df.dtypes)

df = train_df.astype({'target_text':'int'})

print(df.dtypes)

source_text    object
target_text    object
dtype: object
source_text    object
target_text     int64
dtype: object


In [19]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

train_df = train_df.astype({'target_text':'int'})
test_df = test_df.astype({'target_text':'int'})

# Convert other data types to torch.Tensor
train_labels = torch.tensor(train_df.target_text.values)
val_labels = torch.tensor(test_df.target_text.values)

print(train_inputs.shape)
print(train_masks.shape)
print(train_labels.shape)

# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 32

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

torch.Size([9500, 64])
torch.Size([9500, 64])
torch.Size([9500])


In [20]:
%%time
import torch
import torch.nn as nn
from transformers import BertModel

# Create the BertClassfier class
class BertClassifier(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, freeze_bert=False):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module classifier
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        """
        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 50, 2

        # Instantiate BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Instantiate an one-layer feed-forward classifier
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            #nn.Dropout(0.5),
            nn.Linear(H, D_out)
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits

CPU times: user 4.11 ms, sys: 1.03 ms, total: 5.15 ms
Wall time: 6.39 ms


In [21]:
from transformers import AdamW, get_linear_schedule_with_warmup

def initialize_model(epochs=4):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier(freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [22]:
import random
import time

# Specify loss function
loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids, b_attn_mask)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20 batches
            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")
    
    print("Training complete!")


def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

In [24]:
set_seed(42)    # Set seed for reproducibility
bert_classifier, optimizer, scheduler = initialize_model(epochs=2)
train(bert_classifier, train_dataloader, val_dataloader, epochs=2, evaluation=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   20    |   0.640326   |     -      |     -     |   7.02   
   1    |   40    |   0.521171   |     -      |     -     |   6.78   
   1    |   60    |   0.513046   |     -      |     -     |   7.09   
   1    |   80    |   0.460407   |     -      |     -     |   6.92   
   1    |   100   |   0.470741   |     -      |     -     |   6.87   
   1    |   120   |   0.435895   |     -      |     -     |   6.82   
   1    |   140   |   0.434137   |     -      |     -     |   6.81   
   1    |   160   |   0.455721   |     -      |     -     |   6.80   
   1    |   180   |   0.429530   |     -      |     -     |   6.78   
   1    |   200   |   0.413919   |     -      |     -     |   6.75   
   1    |   220   |   0.437248   |     -      |     -     |   6.74   
   1    |   240   |   0.423309   |     -      |     -     |   6.72   


In [25]:
import torch.nn.functional as F

def bert_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate probabilities
    probs = F.softmax(all_logits, dim=1).cpu().numpy()

    return probs

In [26]:
# Compute predicted probabilities on the test set
probs = bert_predict(bert_classifier, val_dataloader)

In [67]:
from sklearn.metrics import accuracy_score, roc_curve, auc, f1_score

preds = probs[:,1]
# Get accuracy over the test set
y_pred = np.where(preds >= 0.5, 1, 0)

accuracy = accuracy_score(val_labels, y_pred)
print(f'Accuracy: {accuracy*100:.2f}%')

# y_pred = torch.tensor([y_pred], dtype=torch.int64)
# print(type(y_pred))
# print(type(val_labels))

F1 = f1_score(val_labels, y_pred)
print('F1 Score:', F1)

Accuracy: 85.60%
F1 Score: 0.8604651162790699


In [68]:
# a = torch.tensor(y_pred)
# print(type(a))
# print(y_pred)
# print(a)
# print(val_labels.shape)
# print(y_pred.shape)

In [69]:
# test_data.sample(5)

In [70]:
# # Run `preprocessing_for_bert` on the test set
# print('Tokenizing data...')
# test_inputs, test_masks = preprocessing_for_bert(test_data.tweet)

# # Create the DataLoader for our test set
# test_dataset = TensorDataset(test_inputs, test_masks)
# test_sampler = SequentialSampler(test_dataset)
# test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=32)

In [71]:
# # Compute predicted probabilities on the test set
# probs = bert_predict(bert_classifier, test_dataloader)

# # Get predictions from the probabilities
# threshold = 0.9
# preds = np.where(probs[:, 1] > threshold, 1, 0)

# # Number of tweets predicted non-negative
# print("Number of tweets predicted non-negative: ", preds.sum())

In [72]:
# print(len(np.where(preds==1)[0]))
# print(len(np.where(preds==0)[0]))

In [73]:
# output = test_data[preds==1]
# list(output.sample(20).tweet)